In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import itertools

In [2]:
training_set_n = pd.read_csv("boston_train.csv").values
test_set_n = pd.read_csv("boston_test.csv").values
prediction_set_n = pd.read_csv("boston_predict.csv").values
np.round (training_set_n[:5])

array([[  2.,   0.,  20.,   1.,   6.,  96.,   2., 403.,  15.,  24.,  nan,
         nan],
       [ 13.,   0.,  18.,   1.,   6.,  95.,   2., 666.,  20.,  13.,  nan,
         nan],
       [  0.,   0.,   7.,   0.,   7.,   3.,   6., 233.,  18.,  27.,  nan,
         nan],
       [  0.,   0.,  11.,   0.,   6.,  18.,   5., 305.,  19.,  22.,  nan,
         nan],
       [  0.,  80.,   2.,   0.,   7.,  38.,   7., 329.,  13.,  35.,  nan,
         nan]])

In [9]:
def prepare_data(df):     
        X_train = df[:, :-3]    
        y_train = df[:,-3]    
        return X_train, y_train	
X_train, y_train = prepare_data(training_set_n)
X_test, y_test = prepare_data(test_set_n)	
x_predict = prediction_set_n[:, :-2]			
print(X_train.shape, y_train.shape, x_predict.shape)			

feature_columns = [tf.feature_column.numeric_column('x', shape=X_train.shape[1:])]
print(feature_columns)

(400, 9) (400,) (6, 9)
[NumericColumn(key='x', shape=(9,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [10]:
estimator = tf.estimator.LinearRegressor(    
         feature_columns=feature_columns,    
         model_dir="train1")	

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'train1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6547d78e48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# Train the estimator
train_input = tf.estimator.inputs.numpy_input_fn(x={"x": X_train},    
                                                 y=y_train,
                                                 batch_size=128,
                                                 shuffle=False,
                                                 num_epochs=None)

In [12]:
estimator.train(input_fn = train_input,steps=5000) 

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into train1/model.ckpt.


In [0]:
eval_input = tf.estimator.inputs.numpy_input_fn(    
       x={"x": X_test},    
       y=y_test, 
       shuffle=False,    
       batch_size=128,    
       num_epochs=1)

In [14]:
estimator.evaluate(eval_input,steps=None)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-03-01T19:42:11Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train1/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-03-01-19:42:11
INFO:tensorflow:Saving dict for global step 5000: average_loss = 19.181934, global_step = 5000, label/mean = 22.08, loss = 1918.1934, prediction/mean = 22.976862
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: train1/model.ckpt-5000


{'average_loss': 19.181934,
 'global_step': 5000,
 'label/mean': 22.08,
 'loss': 1918.1934,
 'prediction/mean': 22.976862}

In [0]:
test_input = tf.estimator.inputs.numpy_input_fn(    
        x={"x": x_predict},    
        batch_size=128,    
        num_epochs=1,   
        shuffle=False)

In [16]:
y = estimator.predict(test_input) 			
predictions = list(p["predictions"] for p in itertools.islice(y, 6))
print("Predictions: {}".format(str(predictions)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train1/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predictions: [array([35.231094], dtype=float32), array([19.068836], dtype=float32), array([24.466223], dtype=float32), array([33.787697], dtype=float32), array([14.434947], dtype=float32), array([20.162895], dtype=float32)]
